Jupyter Notebook used for the cleanup and the formatting of the generated zero-knowledge proofs, allowing the training of the model on it. This notebook must be used before 'model_training_and_evaluation``

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os, os.path

# Name of the folder containing all the generated proofs
folder_name = f"run"

In [ ]:
# First Cleanup, concatenating all the generated proof in one folder and making the first formatting
DIR = f"../typescript/credentials/{folder_name}"
files = [name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]
files.sort()
files = filter(lambda x: ".csv" in x, files)
for name in files:
    df = pd.read_csv(os.path.join(DIR, name))

    # If the proof came from a driving license credential, 
    # we need to recover the total number of attributes of the credential
    if '"Number of categories"' in df.columns:
        df["number_attributes"]= (12
                +df["Number of general additional informations"]
                +3*df["Number of categories"]
                +df["Number of additional inforations of each categories"]).apply(lambda x: sum(map(int, x.split(','))))

    # Store all the data into the temporary csv file before the final cleanup
    df.to_csv(f"cleanup_{folder_name}.csv", mode= "a", header= (not os.path.exists(f"cleanup_{folder_name}.csv")), index = False)
    print(name)


In [ ]:
# Final cleanup, separating the proof into multiple columns
df = pd.read_csv(f"cleanup_{folder_name}.csv")
df["New_Proof"] = df["proof array"].apply(lambda x: list(map(int, x.split(','))))
df["Size"] = df["New_Proof"].apply(lambda x: len(x))

max_length = df["Size"].max()
print("Max length: ", max_length)
for x in range(max_length):
    df["proof_array_"+str(x)] = df["New_Proof"].apply(lambda y: y[x] if len(y) > x else 0)
df = df.drop("New_Proof", axis= 1)
df = df.drop("proof array", axis= 1)
# Final cleaned file
df.to_csv(f"formatted_{folder_name}.csv", index=False)
print(f"formatted_{folder_name}.csv end")